In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot
import matplotlib
matplotlib.style.use('seaborn')
matplotlib.rc('axes', grid=True)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import lore
from my_app.models.product_popularity import Keras
model = Keras.load()

In [ ]:
test = model.pipeline.encoded_test_data.x.copy()
test['response'] = model.pipeline.test_data[model.pipeline.output_encoder.column]
test['color'] = 'gold'

predict = model.pipeline.encoded_test_data.x.copy()
predict['response'] = model.predict(model.pipeline.test_data)
predict['color'] = 'blue'

data = test.append(predict)

def plot_encoder(name):
    stats = data.groupby(['color', name]).agg({'response': ['mean', 'size']}).reset_index()
    stats.columns = ['color', name, 'response', 'population']
    stats['population'] = (stats['population'] / stats['population'].max() * 1000).clip(lower=3)
    stats.plot.scatter(x=name, y='response', s=stats['population'], figsize=(16, 9), c=stats['color'], alpha=0.75)    

for encoder in model.pipeline.encoders:
    if hasattr(encoder, 'sequence_length'):
        for i in range(encoder.sequence_length):
            plot_encoder(encoder.sequence_name(i))
    else:
        plot_encoder(encoder.name)